In [1]:
import pandas as pd
import numpy as np
import plotly.express as px # gráficos dinâmicos
import plotly.graph_objects as go # concatenização de gráficos
from sklearn.preprocessing import StandardScaler # padronização de dados
from sklearn.cluster import KMeans  # Calcula Clusters e WCSS

In [2]:
# abrindo o arquivo
base_iris = pd.read_csv('kmeans_iris.csv', sep = ';')

In [3]:
base_iris.head()

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
base_iris.shape

(150, 5)

In [5]:
# verifica classes das flores
base_iris['class'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

# Realizando o agrupamento pelas pétalas

In [6]:
# variável com colunas "petal length" e "petal width"
petal = base_iris.iloc[:, [2, 3]].values
petal[:10]

array([[1.4, 0.2],
       [1.4, 0.2],
       [1.3, 0.2],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.7, 0.4],
       [1.4, 0.3],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.5, 0.1]])

# Normalizar os dados
Como os dados estão em uma escala muito diferente, recisamos normalizar os dados colocando no mesmo padrão e na mesma escala. Como o Kmeans realiza os cálculos baseado na distância, devemos padronizar os dados afim de que um atributo não seja considerado mais importante do que o outro.

In [7]:
ss = StandardScaler()
petal = ss.fit_transform(petal)
petal[:10]

array([[-1.34022653, -1.3154443 ],
       [-1.34022653, -1.3154443 ],
       [-1.39706395, -1.3154443 ],
       [-1.2833891 , -1.3154443 ],
       [-1.34022653, -1.3154443 ],
       [-1.16971425, -1.05217993],
       [-1.34022653, -1.18381211],
       [-1.2833891 , -1.3154443 ],
       [-1.34022653, -1.3154443 ],
       [-1.2833891 , -1.44707648]])

# Calculando o número de Cluster
Para calcular o número de clusters vamos utilizar o método do cotovelo. WCSS é a soma da distância quadradática entre cada ponto e o seu centróide.

In [8]:
wcss_petal = []
for i in range(1, 11):
  km_petal = KMeans(n_clusters= i, random_state= 0)
  km_petal.fit(petal)
  wcss_petal.append(km_petal.inertia_)

In [9]:
# visualizar valores do wcss
for i in range(len(wcss_petal)):
  print(f'cluster: {i} => valor do wcss: {wcss_petal[i]}')

cluster: 0 => valor do wcss: 300.0
cluster: 1 => valor do wcss: 54.16878133149558
cluster: 2 => valor do wcss: 18.026962612544075
cluster: 3 => valor do wcss: 12.283372197379126
cluster: 4 => valor do wcss: 9.152134379409805
cluster: 5 => valor do wcss: 7.187603589633753
cluster: 6 => valor do wcss: 5.994729609966573
cluster: 7 => valor do wcss: 5.144963679454988
cluster: 8 => valor do wcss: 4.4070793879038
cluster: 9 => valor do wcss: 3.896439832964689


# Gráfico para melhor visualização

In [10]:
gf_cotovelo_petal = px.line(x= range(1, 11), y = wcss_petal)
gf_cotovelo_petal.show()

# Escolher a quantidade de clusters
a quantidade de clusters ideal calculada é o início da estabilização da queda do wcss. E esse valor será definido para a execução do algorítmo.

In [12]:
km_petal = KMeans(n_clusters= 3, random_state= 0)
labels_clustes_petal = km_petal.fit_predict(petal)

In [13]:
# verifica a classificação dos clusters
labels_clustes_petal

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [14]:
# centroides dos clusters
centroides_petal = km_petal.cluster_centers_
centroides_petal

array([[-1.30498732, -1.25489349],
       [ 1.02799959,  1.12797813],
       [ 0.3058728 ,  0.16541778]])

# Gráfico de agrupamento das características do tamanho e comprimento das pétalas

In [15]:
gf_petal = px.scatter(x = petal[:, 0], y = petal[:, 1], color=labels_clustes_petal)
gf_centroide_petal = px.scatter(x = centroides_petal[:, 0], y = centroides_petal[:, 1], size= [7, 7, 7])
gf_final_petal = go.Figure(data = gf_petal.data + gf_centroide_petal.data)
gf_final_petal.show()

# Agrupamento com dados da Sépalas

In [16]:
sepal = base_iris.iloc[:, [0, 1]].values
sepal[:10]

array([[5.1, 3.5],
       [4.9, 3. ],
       [4.7, 3.2],
       [4.6, 3.1],
       [5. , 3.6],
       [5.4, 3.9],
       [4.6, 3.4],
       [5. , 3.4],
       [4.4, 2.9],
       [4.9, 3.1]])

In [17]:
wcss_sepal = []
for i in range(1, 11):
  km_sepal = KMeans(n_clusters= i, random_state= 0)
  km_sepal.fit(sepal)
  wcss_sepal.append(km_sepal.inertia_)

In [18]:
for i in range(len(wcss_sepal)):
  print(f'cluster: {i} => valor do wcss: {wcss_sepal[i]}')

cluster: 0 => valor do wcss: 130.4752666666667
cluster: 1 => valor do wcss: 58.20409278906672
cluster: 2 => valor do wcss: 37.05070212765958
cluster: 3 => valor do wcss: 27.990212038303696
cluster: 4 => valor do wcss: 21.004660660660665
cluster: 5 => valor do wcss: 17.33982216563924
cluster: 6 => valor do wcss: 14.756476273726276
cluster: 7 => valor do wcss: 12.788763481527548
cluster: 8 => valor do wcss: 11.053846569074203
cluster: 9 => valor do wcss: 9.638851368076375


In [19]:
gf_cotovelo_sepal = px.line(x= range(1, 11), y = wcss_sepal)
gf_cotovelo_sepal.show()

In [20]:
km_sepal = KMeans(n_clusters= 3, random_state= 0)
labels_clustes_sepal = km_sepal.fit_predict(sepal)

In [21]:
labels_clustes_sepal

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2,
       2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0])

In [22]:
centroides_sepal = km_sepal.cluster_centers_
centroides_sepal

array([[5.77358491, 2.69245283],
       [5.006     , 3.428     ],
       [6.81276596, 3.07446809]])

In [23]:
# gráfico sem a normalização
gf_sepal = px.scatter(x = sepal[:, 0], y = sepal[:, 1], color=labels_clustes_sepal)
gf_centroide_sepal = px.scatter(x = centroides_sepal[:, 0], y = centroides_sepal[:, 1], size= [7, 7, 7])
gf_final_sepal = go.Figure(data = gf_sepal.data + gf_centroide_sepal.data)
gf_final_sepal.show()

In [24]:
ss = StandardScaler()
sepal = ss.fit_transform(sepal)
labels_clustes_sepal = km_sepal.fit_predict(sepal)
centroides_sepal = km_sepal.cluster_centers_
gf_sepal = px.scatter(x = sepal[:, 0], y = sepal[:, 1], color=labels_clustes_sepal)
gf_centroide_sepal = px.scatter(x = centroides_sepal[:, 0], y = centroides_sepal[:, 1], size= [7, 7, 7])
gf_final_sepal = go.Figure(data = gf_sepal.data + gf_centroide_sepal.data)
gf_final_sepal.show()